In [1]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


import pandas as pd
from IPython.display import display
import json

In [2]:
"""
Change pandas dataframe option.
"""

pd.set_option('display.max_colwidth', None)

In [3]:
"""
Help functions:
- Convert json string to DataFrame.
- Convert Dict to DataFrame.
- Display DataFrame as a Table using IPython library.

"""


def jsonStringToDataFrame(jsonString):
    """
    Convert jsonString to dict.
    """
    jsonStr = str(jsonString)
    strAsJson = json.loads(jsonStr)
    
    # Use pandas.DataFrame.from_dict() to Convert JSON to DataFrame
    dictObj = pd.DataFrame.from_dict(strAsJson, orient="index")
    return dictObj

def dictToDataFrame(Dict):
     # Use pandas.DataFrame.from_dict() to Convert dict to DataFrame
    dictObj = pd.DataFrame.from_dict(Dict, orient="index")
    return dictObj

def displaydf(df):
    """
    Display df as a table
    """
    display(df)

In [4]:
"""
Set the API Scoopes
"""
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly',
          'https://www.googleapis.com/auth/calendar.events',
          'https://www.googleapis.com/auth/calendar']

In [5]:
"""
#Get_events from user calendar.

Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# displaydf(jsonStringToDataFrame(creds.to_json()))


In [6]:
print(creds.expiry)
print(creds.granted_scopes)

2022-12-10 16:15:06
None


In [7]:
"""
If his is the first time the user uses this service, 
let him login and ask for needed permissions,
then save his token. 
"""
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())


In [8]:
"""
The different between printing the base creds and the converted one. 
"""
print("Before:\n")
print(creds.to_json())
print("\n\nAfter:\n")
displaydf(jsonStringToDataFrame(creds.to_json()))

Before:

{"token": "ya29.a0AeTM1icafdtU8NaGjtB4HAP84dmGGXTJ687fT_e0mkEmwY6GBMiASc7wZWbNwgNvqb_cQDulP92sLiyu_A1g_V0uJyle9iELFZe_PidTmxOvUlaJczYJeVY2Mnv8romXKrz62lgo4mp6SC24B-71OxtipJ1oaCgYKATASARISFQHWtWOmISBHm-VCCKpWSfew34EqdA0163", "refresh_token": "1//03FmpQrqFjPMZCgYIARAAGAMSNwF-L9IrvjCM7tuznROiSijyFrlz_xP_04FjP1lQk2jG9li5RLQwZsoXCfTgXZZKCrefv--bz3s", "token_uri": "https://oauth2.googleapis.com/token", "client_id": "344551009993-h76vgmk9rrbctu3i3lbjjs6phoqmt958.apps.googleusercontent.com", "client_secret": "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", "scopes": ["https://www.googleapis.com/auth/calendar.readonly", "https://www.googleapis.com/auth/calendar.events", "https://www.googleapis.com/auth/calendar"], "expiry": "2022-12-10T16:15:06Z"}


After:



,0
token,ya29.a0AeTM1icafdtU8NaGjtB4HAP84dmGGXTJ687fT_e0mkEmwY6GBMiASc7wZWbNwgNvqb_cQDulP92sLiyu_A1g_V0uJyle9iELFZe_PidTmxOvUlaJczYJeVY2Mnv8romXKrz62lgo4mp6SC24B-71OxtipJ1oaCgYKATASARISFQHWtWOmISBHm-VCCKpWSfew34EqdA0163
refresh_token,1//03FmpQrqFjPMZCgYIARAAGAMSNwF-L9IrvjCM7tuznROiSijyFrlz_xP_04FjP1lQk2jG9li5RLQwZsoXCfTgXZZKCrefv--bz3s
token_uri,https://oauth2.googleapis.com/token
client_id,344551009993-h76vgmk9rrbctu3i3lbjjs6phoqmt958.apps.googleusercontent.com
client_secret,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
scopes,"[https://www.googleapis.com/auth/calendar.readonly, https://www.googleapis.com/auth/calendar.events, https://www.googleapis.com/auth/calendar]"
expiry,2022-12-10T16:15:06Z


In [9]:

"""
Get the 
"""

try:
    service = build('calendar', 'v3', credentials=creds)# set the api parameter

    # Call the Calendar API 
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    print('Getting the upcoming 10 events:\n')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                          maxResults=10, singleEvents=True,
                                          orderBy='startTime').execute()#get the next 10 events.
   
    events = events_result.get('items', [])# save the events in a list.

    if not events:
        print('No upcoming events found.')
        

    # Prints the start and name of the next 10 events, and write it to an excel sheet.
    startRow=0
    destination = 'eventsLog.xlsx'
    writer = pd.ExcelWriter(destination, engine='openpyxl')
    
    for event in events:
        print(type(event))
        eventDF = dictToDataFrame(event)
        start = event['start'].get('dateTime', event['start'].get('date'))
        ev = event['summary']
        print(f'Event {ev}')
        sheetname = event['summary']
        eventDF.to_excel(writer, sheet_name= sheetname[:5])
        startRow+=1
        displaydf(eventDF)
        print(start,' ', event['summary'],'\n')
    writer.save()   
except HttpError as error:
    print('An error occurred: %s' % error)

# Refer to the Python quickstart on how to setup the environment:
# https://developers.google.com/calendar/quickstart/python
# Change the scope to 'https://www.googleapis.com/auth/calendar' and delete any
# stored credentials.

Getting the upcoming 10 events:

<class 'dict'>
Event Google I/O 2022{i}


,0
kind,calendar#event
etag,"""3341221427748000"""
id,08mfmtal9m0abceffj8n2blers_20221210T160000Z
status,confirmed
htmlLink,https://www.google.com/calendar/event?eid=MDhtZm10YWw5bTBhYmNlZmZqOG4yYmxlcnNfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
created,2022-12-09T18:31:53.000Z
updated,2022-12-09T18:31:53.874Z
summary,Google I/O 2022{i}
description,A chance to hear more about Google's developer products.
location,"800 Howard St., San Francisco, CA 94103"


2022-12-10T19:00:00+03:00   Google I/O 2022{i} 

<class 'dict'>
Event Google I/O 2022{i}


,0
kind,calendar#event
etag,"""3341221426790000"""
id,1qcds6flcenr9okbk26h6afiak_20221210T160000Z
status,confirmed
htmlLink,https://www.google.com/calendar/event?eid=MXFjZHM2ZmxjZW5yOW9rYmsyNmg2YWZpYWtfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
created,2022-12-09T18:31:53.000Z
updated,2022-12-09T18:31:53.395Z
summary,Google I/O 2022{i}
description,A chance to hear more about Google's developer products.
location,"800 Howard St., San Francisco, CA 94103"


2022-12-10T19:00:00+03:00   Google I/O 2022{i} 

<class 'dict'>
Event Testing calendar API No:1


...

,0
kind,calendar#event
etag,"""3341222161412000"""
id,9b2tqbvss7vnpe5gjgng63rog4_20221210T160000Z
status,confirmed
htmlLink,https://www.google.com/calendar/event?eid=OWIydHFidnNzN3ZucGU1Z2pnbmc2M3JvZzRfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
created,2022-12-09T18:38:00.000Z
updated,2022-12-09T18:38:00.706Z
summary,Testing calendar API No:3
description,A chance to hear more about Google's developer products.
location,"800 Howard St., San Francisco, CA 94103"


2022-12-10T19:00:00+03:00   Testing calendar API No:3 

<class 'dict'>
Event Testing calendar API No:2


,0
kind,calendar#event
etag,"""3341222160474000"""
id,dafb3m84tdu0dirl9edh1b5vuo_20221210T160000Z
status,confirmed
htmlLink,https://www.google.com/calendar/event?eid=ZGFmYjNtODR0ZHUwZGlybDllZGgxYjV2dW9fMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
created,2022-12-09T18:38:00.000Z
updated,2022-12-09T18:38:00.237Z
summary,Testing calendar API No:2
description,A chance to hear more about Google's developer products.
location,"800 Howard St., San Francisco, CA 94103"


2022-12-10T19:00:00+03:00   Testing calendar API No:2 



C:\Users\M7MD\AppData\Local\Temp\ipykernel_10792\1610452245.py:37: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [10]:


"""
Create event json and isert it to the user calendar.
"""

event = {
'summary': 'Google I/O 2022',
'location': '800 Howard St., San Francisco, CA 94103',
'description': 'A chance to hear more about Google\'s developer products.',
'start': {
'dateTime': '2022-12-10T09:00:00-07:00',
'timeZone': 'America/Los_Angeles',
},
'end': {
'dateTime': '2022-12-11T09:00:00-08:00',
'timeZone': 'America/Los_Angeles',
},
'recurrence': [
'RRULE:FREQ=DAILY;COUNT=2'
],
'attendees': [
{'email': 'lpage@example.com'},
{'email': 'sbrin@example.com'},
],
'reminders': {
'useDefault': False,
'overrides': [
    {'method': 'email', 'minutes': 24 * 60},
    {'method': 'popup', 'minutes': 10},
],
},
}

eventsList = []
for i in range(5): 
    event = {
            f'summary': 'Testing calendar API No:' + str(i),
            'location': '800 Howard St., San Francisco, CA 94103',
            'description': 'A chance to hear more about Google\'s developer products.',
            'start': {
            'dateTime': '2022-12-10T09:00:00-07:00',
            'timeZone': 'America/Los_Angeles',
            },
            'end': {
            'dateTime': '2022-12-11T09:00:00-08:00',
            'timeZone': 'America/Los_Angeles',
            },
            'recurrence': [
            'RRULE:FREQ=DAILY;COUNT=2'
            ],
            'attendees': [
            {'email': 'lpage@example.com'},
            {'email': 'sbrin@example.com'},
            ],
            'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 24 * 60},
                {'method': 'popup', 'minutes': 10},
            ],
            },
        }
    eventsList.append(event)

for event in eventsList:
    
    event = service.events().insert(calendarId='primary', body=event).execute()
    print ('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=bmFpcjM2OXExczJsM3E3dWtlc2l1NjZvcThfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
Event created: https://www.google.com/calendar/event?eid=cWQ3dDdvM2Q3c2lvY3UxMGYzM2s2ZnMya2NfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
Event created: https://www.google.com/calendar/event?eid=dDJ0ODdva3V2NzZldjMxNWZqZTNiZTdwM2NfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
Event created: https://www.google.com/calendar/event?eid=YTE2b285M3J1bjV0dHUzM2Nycm03Y2MxdmtfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt
Event created: https://www.google.com/calendar/event?eid=OW5qY2M4OG44ZTJuZ2RrcjA4dGM0dWozaDBfMjAyMjEyMTBUMTYwMDAwWiBtb2hhbW1lZC5zLmc3NkBt


In [11]:
"""
in this cell we will query the free time in the user calendar.
"""
the_datetime = '2022-12-20T15:00:00Z'
the_datetime2 = '2022-12-22T16:00:00Z'
calendarId = 'mohammed.s.g76@gmail.com'

#DateTime should be on the RFC33393 standard
def calendarFreeAt(fromDateTime:str, toDateTime:str, calendarId:str):
    """
    This function should return True if the selected time is free.
    """
    body = {
      "timeMin": fromDateTime,
      "timeMax": toDateTime,
      "timeZone": 'UTC',
      "items": [{"id": calendarId}]
    }


    eventsResult = service.freebusy().query(body=body).execute()
    
    displaydf(dictToDataFrame(eventsResult))

    cal_dict = eventsResult[u'calendars']
    if len(cal_dict[calendarId]['busy']) == 0:
        print('this time is free of events')
        return True
    else:
        for cal_name in cal_dict:
            displaydf(dictToDataFrame(cal_dict))
            print(cal_name, cal_dict[cal_name])
        return False
    
calendarFreeAt(the_datetime, the_datetime2, calendarId)


,0
kind,calendar#freeBusy
timeMin,2022-12-20T15:00:00.000Z
timeMax,2022-12-22T16:00:00.000Z
calendars,{'mohammed.s.g76@gmail.com': {'busy': []}}


this time is free of events


True

In [12]:
the_datetime = '2022-12-20T15:00:00Z'
the_datetime2 = '2022-12-22T16:00:00Z'

body = {
  "timeMin": the_datetime,
  "timeMax": the_datetime2,
  "timeZone": 'UTC',
  "items": [{"id": 'mohammed.s.g76@gmail.com'}]
}


eventsResult = service.freebusy().query(body=body).execute()

# displaydf(dictToDataFrame(eventsResult))

cal_dict = eventsResult[u'calendars']
# displaydf(dictToDataFrame(cal_dict))
for cal_name in cal_dict:
    print(cal_name, cal_dict[cal_name]['busy'])
    eventsList = cal_dict[cal_name]['busy']
    print(type(eventsList))
    if len(eventsList) == 0:
        print('free')
event_dict = cal_dict.get('mohammed.s.g76@gmail.com')

print(event_dict.get('busy'))

mohammed.s.g76@gmail.com []
<class 'list'>
free
[]
